In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from catboost import CatBoostClassifier
import pandas as pd
import json
from pandas import json_normalize
from tqdm import tqdm_notebook as tqdm
import numpy as np
import ast
import time

import sys
import os
sys.path.append(os.path.join(sys.path[0], '../'))

from Baseline.core.utils import calculate_roi, get_winner_favorite, combine_df, parse_odds


In [3]:
fighters_df = pd.read_csv("../Baseline/data/0.fighters_raw.csv", index_col=0)
fighters_df["dateOfBirth"] = pd.to_datetime(fighters_df["dateOfBirth"])
fighters_cols = [
    "id",
    "name",
    "weight",
    "height",
    "armSpan",
    "legSwing",
    "weightCategory.id",
    "weightCategory.name",
    "dateOfBirth",
    "country",
    "city",
    "timezone",
]
fighters_df = fighters_df[fighters_cols]
fighters_df.set_index("id", inplace=True)
f_name_dict = fighters_df['name'].to_dict()
f_name_dict

{1: 'Tanner Boser',
 2: 'Giacomo Lemos',
 3: 'Shamil Abdurakhimov',
 4: 'Klidson Abreu',
 5: 'Yoshihiro Akiyama',
 6: 'Jessica Aguilar',
 7: 'Juan Adams',
 8: 'Israel Adesanya',
 9: 'Abu Azaitar',
 10: 'Rostem Akman',
 11: 'Alexandra Albu',
 12: 'Thiago Alves',
 13: 'Warlley Alves',
 14: 'Hector Aldana',
 15: 'Irene Aldana',
 16: 'Sultan Aliev',
 17: 'Arnold Allen',
 18: 'Eddie Alvarez',
 19: 'Joel Alvarez',
 20: 'Iuri Alcantara',
 21: 'Abdul Razak Alhassan',
 22: 'Thomas Almeida',
 23: 'Corey Anderson',
 24: 'Makwan Amirkhani',
 25: 'Eryk Anders',
 26: 'Megan Anderson',
 27: 'Viviane Araujo',
 28: 'Jessica Andrade',
 29: 'Magomed Ankalaev',
 30: 'Gadzhimurad Antigulov',
 31: 'Austin Arnett',
 32: 'Ben Askren',
 33: 'Julio Arce',
 34: 'Cyril Asker',
 35: 'Omari Akhmedov',
 36: 'Jessin Ayari',
 37: 'Bryan Barberena',
 38: 'Renan Barao',
 39: 'Edson Barboza',
 40: 'Enrique Barzola',
 41: 'Mario Bautista',
 42: 'Raoni Barcelos',
 43: 'Siyar Bahadurzada',
 44: 'Gabriel Benitez',
 45: 'Ryan

In [4]:
ready_cols = ast.literal_eval(open('data/ready_cols_05.04.2021.txt', 'r').read())
ready_cols

['cumsumwinner',
 'cumsum_hitsTotal',
 'cumsum_hitsSuccessful',
 'cumsum_takedownTotal',
 'cumsum_takedownSuccessful',
 'cumsum_submissionAttempts',
 'cumsum_takeovers',
 'cumsum_accentedHitsTotal',
 'cumsum_accentedHitsSuccessful',
 'cumsum_knockdowns',
 'cumsum_protectionPassage',
 'cumsum_hitsHeadTotal',
 'cumsum_hitsHeadSuccessful',
 'cumsum_hitsBodyTotal',
 'cumsum_hitsBodySuccessful',
 'cumsum_hitsLegsTotal',
 'cumsum_hitsLegsSuccessful',
 'cumsum_accentedHitsPositionDistanceTotal',
 'cumsum_accentedHitsPositionDistanceSuccessful',
 'cumsum_accentedHitsPositionClinchTotal',
 'cumsum_accentedHitsPositionClinchSuccessful',
 'cumsum_accentedHitsPositionParterTotal',
 'cumsum_accentedHitsPositionParterSuccessful',
 'cumsumwinMethods_[DEC]',
 'cumsumwinMethods_[DQ]',
 'cumsumwinMethods_[KO]',
 'cumsumwinMethods_[SUB]',
 'count_of_fights']

In [5]:
f_stats_events_cumulative = pd.read_csv('data/f_stats_events_cumulative_05.04.2021.csv', index_col=0)
f_stats_events_cumulative

,cumsumduration,cumsumwinner,cumsum_hitsTotal,cumsum_hitsSuccessful,cumsum_takedownTotal,cumsum_takedownSuccessful,cumsum_submissionAttempts,cumsum_takeovers,cumsum_accentedHitsTotal,cumsum_accentedHitsSuccessful,...,cumsumwinMethods_[DQ],cumsumwinMethods_[KO],cumsumwinMethods_[SUB],eventDate.date,fighterId,duration,winner,fighter_nbr,count_of_fights,fighterName
6266,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,2019-10-19 04:00:00,1,300.0,True,1,0,Tanner Boser
6353,300.0,1,51.0,35.0,0.0,0.0,0.0,0.0,51.0,35.0,...,0,0,0,2019-12-21 00:00:00,1,300.0,False,1,1,Tanner Boser
6532,600.0,1,88.0,48.0,0.0,0.0,0.0,0.0,88.0,48.0,...,0,0,0,2020-06-27 00:00:00,1,161.0,True,1,2,Tanner Boser
6585,761.0,2,110.0,63.0,0.0,0.0,0.0,0.0,110.0,63.0,...,0,1,0,2020-07-25 00:00:00,1,156.0,True,1,3,Tanner Boser
6733,917.0,3,146.0,91.0,0.0,0.0,0.0,0.0,146.0,91.0,...,0,2,0,2020-11-07 00:00:00,1,300.0,False,1,4,Tanner Boser
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6820,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,2021-01-20 00:00:00,3664,248.0,True,2,0,Manon Fiorot
6806,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,2021-01-16 00:00:00,3666,300.0,True,2,0,Joselyne Edwards
6836,300.0,1,93.0,42.0,2.0,0.0,0.0,0.0,93.0,42.0,...,0,0,0,2021-02-06 00:00:00,3666,300.0,False,2,1,Joselyne Edwards
6813,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,2021-01-20 00:00:00,3667,240.0,False,2,0,Gaetano Pirrello


In [6]:
f_stats_events_cumulative[f_stats_events_cumulative['fighterId'] == 3667]

,cumsumduration,cumsumwinner,cumsum_hitsTotal,cumsum_hitsSuccessful,cumsum_takedownTotal,cumsum_takedownSuccessful,cumsum_submissionAttempts,cumsum_takeovers,cumsum_accentedHitsTotal,cumsum_accentedHitsSuccessful,...,cumsumwinMethods_[DQ],cumsumwinMethods_[KO],cumsumwinMethods_[SUB],eventDate.date,fighterId,duration,winner,fighter_nbr,count_of_fights,fighterName
6813,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,2021-01-20 00:00:00,3667,240.0,False,2,0,Gaetano Pirrello


In [7]:
f_stats_events_cumulative[f_stats_events_cumulative['fighterName'] == 'Donald Cerrone']

,cumsumduration,cumsumwinner,cumsum_hitsTotal,cumsum_hitsSuccessful,cumsum_takedownTotal,cumsum_takedownSuccessful,cumsum_submissionAttempts,cumsum_takeovers,cumsum_accentedHitsTotal,cumsum_accentedHitsSuccessful,...,cumsumwinMethods_[DQ],cumsumwinMethods_[KO],cumsumwinMethods_[SUB],eventDate.date,fighterId,duration,winner,fighter_nbr,count_of_fights,fighterName
1645,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,2008-11-05 00:00:00,347,300.0,True,1,0,Donald Cerrone
2376,300.0,1,69.0,30.0,3.0,2.0,0.0,0.0,68.0,29.0,...,0,0,0,2010-12-16 00:00:00,347,163.0,True,1,1,Donald Cerrone
2418,463.0,2,74.0,33.0,3.0,2.0,3.0,0.0,72.0,31.0,...,0,0,1,2011-02-05 00:00:00,347,228.0,True,1,2,Donald Cerrone
2533,691.0,3,122.0,64.0,4.0,3.0,4.0,0.0,106.0,48.0,...,0,0,2,2011-06-11 00:00:00,347,300.0,True,1,3,Donald Cerrone
2592,991.0,4,188.0,104.0,5.0,3.0,4.0,0.0,172.0,88.0,...,0,0,2,2011-08-14 00:00:00,347,181.0,True,2,4,Donald Cerrone
2676,1172.0,5,251.0,136.0,5.0,3.0,4.0,0.0,232.0,118.0,...,0,1,2,2011-10-29 00:00:00,347,142.0,True,1,5,Donald Cerrone
2751,1314.0,6,274.0,145.0,5.0,3.0,5.0,0.0,255.0,127.0,...,0,1,3,2011-12-30 00:00:00,347,300.0,False,2,6,Donald Cerrone
2878,1614.0,6,351.0,182.0,5.0,3.0,5.0,0.0,326.0,160.0,...,0,1,3,2012-05-15 00:00:00,347,300.0,True,1,7,Donald Cerrone
2989,1914.0,7,419.0,213.0,6.0,3.0,5.0,0.0,394.0,191.0,...,0,1,3,2012-08-11 00:00:00,347,76.0,True,1,8,Donald Cerrone
3131,1990.0,8,428.0,217.0,6.0,3.0,5.0,0.0,403.0,195.0,...,0,2,3,2013-01-26 00:00:00,347,155.0,False,2,9,Donald Cerrone


In [8]:
import datetime
f1_birthDate = fighters_df[fighters_df['name'] == f1_name]['dateOfBirth']
f1_birthDate 


NameError: name 'f1_name' is not defined

In [9]:
f1_age = ((pd.to_datetime(event_date) - pd.to_datetime(f1_birthDate)) / 365).dt.days.values[0]
f1_age

NameError: name 'event_date' is not defined

In [10]:

f1_age = ((pd.to_datetime(time.ctime()) - pd.to_datetime(f1_birthDate)) / 365).dt.days.values[0]
f1_age

NameError: name 'f1_birthDate' is not defined

In [11]:
f1_name = 'Donald Cerrone'
f2_name = 'Jim Miller'
event_date = datetime.date(2021, 4, 5)


f1_birthDate = fighters_df[fighters_df['name'] == f1_name]['dateOfBirth']
f1_age = ((pd.to_datetime(event_date) - pd.to_datetime(f1_birthDate)) / 365).dt.days.values[0]

f2_birthDate = fighters_df[fighters_df['name'] == f2_name]['dateOfBirth']
f2_age = ((pd.to_datetime(event_date) - pd.to_datetime(f2_birthDate)) / 365).dt.days.values[0]


fighter1_stats = f_stats_events_cumulative[f_stats_events_cumulative['fighterName'] == f1_name]#.iloc[-1]
fighter1_stats = pd.DataFrame(fighter1_stats[fighter1_stats[ready_cols].isna().sum(axis=1) < 10].iloc[-1]).T.reset_index(drop=True)

fighter2_stats = f_stats_events_cumulative[f_stats_events_cumulative['fighterName'] == f2_name]#.iloc[-1]
fighter2_stats = pd.DataFrame(fighter2_stats[fighter2_stats[ready_cols].isna().sum(axis=1) < 10].iloc[-1]).T.reset_index(drop=True)
fighter2_stats

,cumsumduration,cumsumwinner,cumsum_hitsTotal,cumsum_hitsSuccessful,cumsum_takedownTotal,cumsum_takedownSuccessful,cumsum_submissionAttempts,cumsum_takeovers,cumsum_accentedHitsTotal,cumsum_accentedHitsSuccessful,...,cumsumwinMethods_[DQ],cumsumwinMethods_[KO],cumsumwinMethods_[SUB],eventDate.date,fighterId,duration,winner,fighter_nbr,count_of_fights,fighterName
0,7365.0,20,972.0,451.0,40.0,18.0,18.0,0.0,829.0,323.0,...,0,5,12,2020-06-20 00:00:00,229,145.0,True,1,33,Jim Miller


In [13]:
X_df = pd.DataFrame(index=[0])

X_df = X_df.join(
    fighter1_stats[ready_cols].add_prefix("f1_")) #, on="id"

X_df = X_df.join(
    fighter2_stats[ready_cols].add_prefix("f2_"))

X_df.loc[0, ['f1_age', 'f2_age']] = f1_age, f2_age
X_df

,f1_cumsumwinner,f1_cumsum_hitsTotal,f1_cumsum_hitsSuccessful,f1_cumsum_takedownTotal,f1_cumsum_takedownSuccessful,f1_cumsum_submissionAttempts,f1_cumsum_takeovers,f1_cumsum_accentedHitsTotal,f1_cumsum_accentedHitsSuccessful,f1_cumsum_knockdowns,...,f2_cumsum_accentedHitsPositionClinchSuccessful,f2_cumsum_accentedHitsPositionParterTotal,f2_cumsum_accentedHitsPositionParterSuccessful,f2_cumsumwinMethods_[DEC],f2_cumsumwinMethods_[DQ],f2_cumsumwinMethods_[KO],f2_cumsumwinMethods_[SUB],f2_count_of_fights,f1_age,f2_age
0,25,1553.0,798.0,32.0,11.0,10.0,1.0,1461.0,718.0,0.0,...,54.0,65.0,42.0,16,0,5,12,33,38.0,37.0


In [14]:
def difference(events_df_joined, cols):
    df = pd.DataFrame()
    # цикл заменяет столбцы характеристик каждого бойца столбцами разницы этих характеристик
    for col in cols:
        df[col+'_difference'] = events_df_joined['f1_'+col].astype(float) - events_df_joined['f2_'+col].astype(float)

    df['age_difference'] = events_df_joined['f1_age'] - events_df_joined['f2_age'] # не стал удалять столбцы с возрастом

    return df

X_df_combined = difference(X_df, ready_cols)
X_df_combined_reversed = X_df_combined*-1
X_df_combined_reversed

,cumsumwinner_difference,cumsum_hitsTotal_difference,cumsum_hitsSuccessful_difference,cumsum_takedownTotal_difference,cumsum_takedownSuccessful_difference,cumsum_submissionAttempts_difference,cumsum_takeovers_difference,cumsum_accentedHitsTotal_difference,cumsum_accentedHitsSuccessful_difference,cumsum_knockdowns_difference,...,cumsum_accentedHitsPositionClinchTotal_difference,cumsum_accentedHitsPositionClinchSuccessful_difference,cumsum_accentedHitsPositionParterTotal_difference,cumsum_accentedHitsPositionParterSuccessful_difference,cumsumwinMethods_[DEC]_difference,cumsumwinMethods_[DQ]_difference,cumsumwinMethods_[KO]_difference,cumsumwinMethods_[SUB]_difference,count_of_fights_difference,age_difference
0,-5.0,-581.0,-347.0,8.0,7.0,8.0,-1.0,-632.0,-395.0,-0.0,...,-13.0,-11.0,-47.0,-40.0,4.0,-0.0,-12.0,5.0,-3.0,-1.0


In [15]:
clf = CatBoostClassifier()

clf.load_model('models/catboost_05.04.2021.cat')
model_cols = clf.feature_names_
model_cols

['cumsumwinner_difference',
 'cumsum_hitsTotal_difference',
 'cumsum_hitsSuccessful_difference',
 'cumsum_takedownTotal_difference',
 'cumsum_takedownSuccessful_difference',
 'cumsum_submissionAttempts_difference',
 'cumsum_takeovers_difference',
 'cumsum_accentedHitsTotal_difference',
 'cumsum_accentedHitsSuccessful_difference',
 'cumsum_knockdowns_difference',
 'cumsum_protectionPassage_difference',
 'cumsum_hitsHeadTotal_difference',
 'cumsum_hitsHeadSuccessful_difference',
 'cumsum_hitsBodyTotal_difference',
 'cumsum_hitsBodySuccessful_difference',
 'cumsum_hitsLegsTotal_difference',
 'cumsum_hitsLegsSuccessful_difference',
 'cumsum_accentedHitsPositionDistanceTotal_difference',
 'cumsum_accentedHitsPositionDistanceSuccessful_difference',
 'cumsum_accentedHitsPositionClinchTotal_difference',
 'cumsum_accentedHitsPositionClinchSuccessful_difference',
 'cumsum_accentedHitsPositionParterTotal_difference',
 'cumsum_accentedHitsPositionParterSuccessful_difference',
 'cumsumwinMethods_[D

In [17]:
y_proba_straight = clf.predict_proba(X_df_combined[model_cols])[:,1]
y_proba_reversed = clf.predict_proba(X_df_combined_reversed[model_cols])[:,0]
y_proba = np.mean([y_proba_straight, y_proba_reversed])
y_proba

0.44642076214167725